In [1]:
#Model
import numpy as np
import pandas as pd
from keras.layers import Dense,Flatten,Conv2D
from keras.layers import MaxPooling2D,Dropout
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical


In [2]:
#importing data
data=pd.read_csv("train_foo.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:, 1:2501]
Y=Y[:, 0]
X=X/255




In [3]:
#splitting into train and test
n_classes = 12
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)


In [4]:
print("number of training samples = "+str(x_train.shape[0]))
print("number of test samples = "+str(x_test.shape[0]))
print("X_train shape:"+str(x_train.shape))
print("X_test shape:"+str(x_test.shape))
print("Y_train shape:"+str(y_train.shape))
print("Y_test shape:"+str(y_test.shape))

number of training samples = 1488
number of test samples = 639
X_train shape:(1488, 2500)
X_test shape:(639, 2500)
Y_train shape:(1488, 12)
Y_test shape:(639, 12)


In [5]:
image_x=50
image_y=50
im_shape = (image_x, image_y, 1)
train_y=y_train.reshape(y_train.shape[0],y_train.shape[1])
test_y=y_test.reshape(y_test.shape[0],y_test.shape[1])
x_train=x_train.reshape(x_train.shape[0],50,50,1)
x_test=x_test.reshape(x_test.shape[0],50,50,1)
print("X_train shape:"+str(x_train.shape))
print("Y_train shape:"+str(train_y.shape))
print("X_test shape:"+str(x_test.shape))
print("Y_test shape:"+str(test_y.shape))
x_train= np.asarray(x_train).astype(np.float32)
x_test=np.asarray(x_test).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
test_y=np.asarray(test_y).astype(np.float32)



X_train shape:(1488, 50, 50, 1)
Y_train shape:(1488, 12)
X_test shape:(639, 50, 50, 1)
Y_test shape:(639, 12)


In [6]:
def keras_model(image_x,image_y):
    n_classes=12
    model=Sequential()
    model.add(Conv2D(32,(5,5),input_shape=(im_shape),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(n_classes,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="emojinator.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return model,callbacks_list
    
    
    


In [ ]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(x_train,train_y,validation_data=(x_test,test_y),epochs=10,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(x_test,test_y,verbose=0)
print("CNN Error:%.2f%%"%(100-scores[1]*100))
model.summary()
model.save('emojinator.h5')

Epoch 1/10
24/24 [==============================] - 18s 742ms/step - loss: 0.7413 - accuracy: 0.7742 - val_loss: 8.5964e-04 - val_accuracy: 1.0000
Epoch 2/10
24/24 [==============================] - 17s 696ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 6.1591e-06 - val_accuracy: 1.0000
Epoch 3/10
24/24 [==============================] - 17s 703ms/step - loss: 7.6594e-05 - accuracy: 1.0000 - val_loss: 1.6721e-06 - val_accuracy: 1.0000
Epoch 4/10
24/24 [==============================] - 17s 711ms/step - loss: 3.9613e-05 - accuracy: 1.0000 - val_loss: 1.1712e-06 - val_accuracy: 1.0000
Epoch 5/10
16/24 [===================>..........] - ETA: 5s - loss: 5.2539e-05 - accuracy: 1.0000